In [ ]:
from rdkit import Chem
import pandas as pd

SMRT_dataset_sdf = Chem.SDMolSupplier("SMRT_dataset.sdf") 
SMRT_dataset_df = pd.read_csv("SMRT_dataset.csv",delimiter=';')


In [ ]:
from tqdm import tqdm
id_smiles_dict = {}
for mol in tqdm(SMRT_dataset_sdf):
    if mol is not None:
        id_smiles_dict[int(mol.GetProp("_Name"))] = Chem.CanonSmiles(Chem.MolToSmiles(mol))

In [ ]:
import pubchempy as pcp

for idx,row in SMRT_dataset_df.iterrows():
    if pd.isnull(row.SMILES):
        print(f"Missing SMILES for pubchem ID: {row.pubchem}")
        compound = pcp.Compound.from_cid(row.pubchem)
        id_smiles_dict[row.pubchem] = Chem.CanonSmiles(compound.smiles)

In [ ]:
SMRT_dataset_df.loc[:, "SMILES"] = SMRT_dataset_df.pubchem.map(id_smiles_dict)
SMRT_dataset_df.SMILES.isnull().sum()

def remove_atom_map_numbers(s):
    mol = Chem.MolFromSmiles(s)
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(0)
    return Chem.CanonSmiles(Chem.MolToSmiles(mol))
SMRT_dataset_df.loc[:, "SMILES"] = SMRT_dataset_df.loc[:, "SMILES"].apply(remove_atom_map_numbers)

np.int64(0)

In [32]:
SMRT_dataset_df.to_csv("SMRT_dataset_processed.csv",index=False)